In [3]:
from openpyxl import load_workbook
from tqdm.notebook import tqdm
import json
import re

In [8]:
def get_connected_reference(company, resource, key, value):
    value=value.strip()
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [9]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)
    if field == 'Nome da placa':
        value=value.upper()
    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [4]:
def load_excel(filename,sheetPosision=0):
    wb = load_workbook(filename=filename+'.xlsx')
    sheet = wb[wb.sheetnames[sheetPosision]]
    data = []

    headers = [cell.value for cell in sheet[1]]

    for row in sheet.iter_rows(min_row=2):
        row_dict = {}
        for key, cell in zip(headers, row):
            row_dict[key] = cell.value
    #         if 'Foto Contenção 1_1' == key:
    #             print(cell.value)
        data.append(row_dict)
    return data

In [5]:
def convert_string(string: str) -> str:
    if '_' in string:
        # Convert snake_case to camelCase
        return re.sub(r'_([a-z])', lambda x: x.group(1).upper(), string)
    else:
        # Convert camelCase to snake_case
        return re.sub(r'([A-Z])', lambda x: '_' + x.group(1).lower(), string)

In [ ]:
values=load_excel('PROJETO',0)

In [18]:
company=Company.objects.get(name='Arteris Via Paulista - SISOBRAS/SISQUALI')
user=User.objects.get(username='rlcs')
occProjeto=OccurrenceType.objects.get(name='Concreto Asfáltico',company=company)
occProjeto,user

(<OccurrenceType: Concreto Asfáltico - ['Arteris Via Paulista - SISOBRAS/SISQUALI']>,
 <User: rlcs - Kartado CS>)

In [7]:
for a in occProjeto.form_fields['fields']:
    if a.get("apiName"):
        print(f'"{convert_string(a["apiName"])}": a["{a["displayName"]}"]{a["dataType"]}')
    else:
        print(f'"{convert_string(a["api_name"])}": a["{a["display_name"]}"]{a["data_type"]}')
    if a.get('dataType') == 'arrayOfObjects':
        for b in a['innerFields']:
            print(f'\t"{convert_string(b["apiName"])}": a["{b["displayName"]}"]{b["dataType"]}')
    if a.get('data_type') == 'arrayOfObjects':
        for b in a['inner_fields']:
            print(f'\t"{convert_string(b["api_name"])}": a["{b["display_name"]}"]{b["data_type"]}')

"range_gran_conc_asf": a["Faixa Granulométrica"]string
"ligante_conc_asf": a["Ligante"]select
"teor_cap_conc_asf": a["Teor de cap (%)"]float
"empty_volume_conc_asf": a["Volume de vazios (%)"]float
"vam_conc_asf": a["VAM (%)"]float
"mass_esp_visi_conc_asf": a["Massa Específca Aparente (g/cm³)"]float
"mass_esp_max_conc_asf": a["Massa Específca Máxima (g/cm³)"]float
"mass_esp_efective_conc_asf": a["Massa Específca Efetiva dos Agregdos (g/cm³)"]float
"aggregate1_conc_asf": a["Agregado 1"]select
"percentage_aggregate1_conc_asf": a["% de Agregado 1"]float
"aggregate2_conc_asf": a["Agregado 2"]select
"percentage_aggregate2_conc_asf": a["% de Agregado 2"]float
"aggregate3_conc_asf": a["Agregado 3"]select
"percentage_aggregate3_conc_asf": a["% de Agregado 3"]float
"aggregate4_conc_asf": a["Agregado 4"]select
"percentage_aggregate4_conc_asf": a["% de Agregado 4"]float
"filer_conc_asf": a["Fíler"]select
"percentage_filer_conc_asf": a["% de Fíler"]float


# Importação Projeto

In [ ]:
objects=[]

for index, a in enumerate(tqdm(values)):
    
    try:
        objects.append(QualityProject(
                    project_number=a['Nº do Projeto'],
                    firm=Firm.objects.get(name=a['Equipe/Construtora'],company=company),
                    occurrence_type=occProjeto,
                    created_at=a['Data de Criação'],
                    registered_at=datetime.today(),
                    expires_at=a['Data de Validade'],
                    form_data={
                        "range_gran_conc_asf": a["Faixa Granulométrica"],
                        "ligante_conc_asf": get_value('Ligante',occurrence_type,a["Ligante"].strip()) if a["Ligante"] is not None else None,
                        "teor_cap_conc_asf": a["Teor de cap (%)"],
                        "empty_volume_conc_asf": a["Volume de vazios (%)"],
                        "vam_conc_asf": a["VAM (%)"],
                        "mass_esp_visi_conc_asf": a["Massa Específca Aparente (g/cm³)"],
                        "mass_esp_max_conc_asf": a["Massa Específca Máxima (g/cm³)"],
                        "mass_esp_efective_conc_asf": a["Massa Específca Efetiva dos Agregdos (g/cm³)"],
                        "aggregate1_conc_asf": get_value('Agregado 1',occurrence_type,a["Agregado 1"].strip()) if a["Agregado 1"] is not None else None,
                        "percentage_aggregate1_conc_asf": a["% de Agregado 1"],
                        "aggregate2_conc_asf": get_value('Agregado 2',occurrence_type,a["Agregado 2"].strip()) if a["Agregado 2"] is not None else None,
                        "percentage_aggregate2_conc_asf": a["% de Agregado 2"],
                        "aggregate3_conc_asf": get_value('Agregado 3',occurrence_type,a["Agregado 3"].strip()) if a["Agregado 3"] is not None else None,
                        "percentage_aggregate3_conc_asf": a["% de Agregado 3"],
                        "aggregate4_conc_asf": get_value('Agregado 4',occurrence_type,a["Agregado 4"].strip()) if a["Agregado 4"] is not None else None,
                        "percentage_aggregate4_conc_asf": a["% de Agregado 4"],
                        "filer_conc_asf": get_value('Fíler',occurrence_type,a["Fíler"].strip()) if a["Fíler"] is not None else None,
                        "percentage_filer_conc_asf": a["% de Fíler"]
                        },
                    ))
    except Exception as e:
        print(f'AQUI: Linha da Planilha: {index+2} erro: {e}')
#         print(objects.form_data)
        pass

In [ ]:
objects[0]

In [ ]:
for projeto in tqdm(objects):
    projeto.save()
    print(projeto.number)

# Amostras

In [13]:
occAmostra = OccurrenceType.objects.get(name='Amostra CBUQ',company=company)
occAmostra

<OccurrenceType: Amostra CBUQ - ['Arteris Via Paulista - SISOBRAS/SISQUALI']>

In [14]:
for a in occAmostra.form_fields['fields']:
    if a.get("apiName"):
        print(f'"{convert_string(a["apiName"])}": a["{a["displayName"]}"]{a["dataType"]}')
    else:
        print(f'"{convert_string(a["api_name"])}": a["{a["display_name"]}"]{a["data_type"]}')
    if a.get('dataType') == 'arrayOfObjects':
        for b in a['innerFields']:
            print(f'\t"{convert_string(b["apiName"])}": a["{b["displayName"]}"]{b["dataType"]}')
    if a.get('data_type') == 'arrayOfObjects':
        for b in a['inner_fields']:
            print(f'\t"{convert_string(b["api_name"])}": a["{b["display_name"]}"]{b["data_type"]}')

"colected_place": a["Local da Coleta"]string
"camada": a["Camada"]string
"certify": a["Certificado"]number
"teste_date": a["Data do ensaio"]timestamp


## Importação Amostras

In [ ]:
objectsAmostra=[]

for index, a in enumerate(tqdm(values)):
    
    try:
        objectsAmostra.append(QualitySample(
                    company=company,
                    collected_at=a['Data da Coleta'],
                    created_at=datetime.today(),
                    created_by=user,
                    responsible=str(User.objects.get(first_name=a['Responsável pela Coleta'].split()[0],
                                                     last_name=a['Responsável pela Coleta'].split()[1]).uuid),
                    quality_project=QualityProject.objects.get(project_number=a['Nº do Projeto']),
                    project_number=a['Nº do Projeto'],
                    construction_firm=Firm.objects.get(name=a['Construtora'],company=company),
                    occurrence_type=occAmostra,
                    construction_plant=ConstructionPlant.objects.get(name=a['Pedreira/Usina'],company=company),
                    received_at=a['Data do Recebimento'],
                    is_proof=True if a['É uma contra prova?'].lower() == 'sim' else False,
                    form_data={
                        "colected_place": a["Local da Coleta"],
                        "camada": a["Camada"],
                        "certify": a["Certificado"],
                        "teste_date": a["Data do ensaio"].strftime('%Y-%m-%dT03:00:00.000Z')
                        },
                    ))
    except Exception as e:
        print(f'AQUI: Linha da Planilha: {index+2} erro: {e}')
#         print(objects.form_data)
        pass

In [ ]:
objectsAmostra[0]

In [ ]:
for amostra in tqdm(objectsAmostra):
    amostra.save()
    print(amostra.number)

# Ensaio

In [23]:
occEnsaio = OccurrenceType.objects.get(name='Ensaio CBUQ',company=company)
occEnsaio

<OccurrenceType: Ensaio CBUQ - ['Arteris Via Paulista - SISOBRAS/SISQUALI']>

In [25]:
for a in occEnsaio.form_fields['fields']:
    if a.get("apiName"):
        print(f'"{convert_string(a["apiName"])}": a["{a["displayName"]}"] if a["{a["displayName"]}"] is not None else None,')
    else:
        print(f'"{convert_string(a["api_name"])}": a["{a["display_name"]}"] if a["{a["displayName"]}"] is not None else None,')
    if a.get('dataType') == 'arrayOfObjects':
        for b in a['innerFields']:
            print(f'\t"{convert_string(b["apiName"])}": a["{b["displayName"]}"] if a["{a["displayName"]}"] is not None else None,')
    if a.get('data_type') == 'arrayOfObjects':
        for b in a['inner_fields']:
            print(f'\t"{convert_string(b["api_name"])}": a["{b["display_name"]}"] if a["{a["displayName"]}"] is not None else None,')

"teor_founded_lig": a["Teor de Ligante Encontrado"] if a["Teor de Ligante Encontrado"] is not None else None,
"teor_founded_lig_lim_min": a["Teor de Ligante Limites Min"] if a["Teor de Ligante Limites Min"] is not None else None,
"teor_founded_lig_lim_max": a["Teor de Ligante Limites Max"] if a["Teor de Ligante Limites Max"] is not None else None,
"factor_consider_correct": a["Fator de correção considerado"] if a["Fator de correção considerado"] is not None else None,
"mass_esp_max_measure": a["Massa Esp. Máxima Medida (Rice)"] if a["Massa Esp. Máxima Medida (Rice)"] is not None else None,
"damage_damp_ind": a["Danos por Umidade Induzida (%RRT)"] if a["Danos por Umidade Induzida (%RRT)"] is not None else None,
"mass_espec_efective_group": a["Massa Específica Efetiva dos Agregados"] if a["Massa Específica Efetiva dos Agregados"] is not None else None,
"passerby_sample2": a["(ASTM 2'')% Passante da Amostra"] if a["(ASTM 2'')% Passante da Amostra"] is not None else None,
"passerby_sample1

## Importação Ensaio

In [26]:
QualityAssay._meta.get_fields()

(<ManyToManyRel: quality_control.qualityassay>,
 <django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.CharField: number>,
 <django.db.models.fields.related.ForeignKey: company>,
 <django.db.models.fields.DateTimeField: created_at>,
 <django.db.models.fields.DateTimeField: executed_at>,
 <django.db.models.fields.related.ForeignKey: created_by>,
 <django.db.models.fields.related.ForeignKey: responsible>,
 <django.db.models.fields.related.ForeignKey: quality_project>,
 <django.db.models.fields.related.ForeignKey: occurrence_type>,
 <django.db.models.fields.related.ForeignKey: quality_sample>,
 <django.contrib.postgres.fields.jsonb.JSONField: form_data>,
 <django.db.models.fields.TextField: notes>,
 <django.db.models.fields.related.ForeignKey: csv_import>,
 <django.db.models.fields.related.ManyToManyField: related_assays>,
 <django.db.models.fields.related.ManyToManyField: reportings>)

In [ ]:
objectsEnsaio=[]

for index, a in enumerate(tqdm(values)):
    
    try:
        objectsEnsaio.append(QualityAssay(
                    company=company,
                    created_at=a['Data de Criação'],
                    created_by=user,
                    responsible=str(User.objects.get(first_name=a['Responsável pela Coleta'].split()[0],
                                                     last_name=a['Responsável pela Coleta'].split()[1]).uuid),
                    quality_project=QualityProject.objects.get(project_number=a['Nº do Projeto']),
                    project_number=a['Nº do Projeto'],
                    form_data={
                        "teor_founded_lig": a["Teor de Ligante Encontrado"] if a["Teor de Ligante Encontrado"] is not None else None,
                        "teor_founded_lig_lim_min": a["Teor de Ligante Limites Min"] if a["Teor de Ligante Limites Min"] is not None else None,
                        "teor_founded_lig_lim_max": a["Teor de Ligante Limites Max"] if a["Teor de Ligante Limites Max"] is not None else None,
                        "factor_consider_correct": a["Fator de correção considerado"] if a["Fator de correção considerado"] is not None else None,
                        "mass_esp_max_measure": a["Massa Esp. Máxima Medida (Rice)"] if a["Massa Esp. Máxima Medida (Rice)"] is not None else None,
                        "damage_damp_ind": a["Danos por Umidade Induzida (%RRT)"] if a["Danos por Umidade Induzida (%RRT)"] is not None else None,
                        "mass_espec_efective_group": a["Massa Específica Efetiva dos Agregados"] if a["Massa Específica Efetiva dos Agregados"] is not None else None,
                        "passerby_sample2": a["(ASTM 2'')% Passante da Amostra"] if a["(ASTM 2'')% Passante da Amostra"] is not None else None,
                        "passerby_sample112": a["(ASTM 1 1/2'') % Passante da Amostra"] if a["(ASTM 1 1/2'') % Passante da Amostra"] is not None else None,
                        "passerby_sample1": a["(ASTM 1'') % Passante da Amostra"] if a["(ASTM 1'') % Passante da Amostra"] is not None else None,
                        "passerby_sample34": a["(ASTM 3/4'') % Passante da Amostra"] if a["(ASTM 3/4'') % Passante da Amostra"] is not None else None,
                        "passerby_sample58": a["(ASTM 5/8'') % Passante da Amostra"] if a["(ASTM 5/8'') % Passante da Amostra"] is not None else None,
                        "passerby_sample12": a["(ASTM 1/2'') % Passante da Amostra"] if a["(ASTM 1/2'') % Passante da Amostra"] is not None else None,
                        "passerby_sample38": a["(ASTM 3/8'') % Passante da Amostra"] if a["(ASTM 3/8'') % Passante da Amostra"] is not None else None,
                        "passerby_sample_n4": a["(ASTM Nº4) % Passante da Amostra"] if a["(ASTM Nº4) % Passante da Amostra"] is not None else None,
                        "passerby_sample_n10": a["(ASTM Nº10) % Passante da Amostra"] if a["(ASTM Nº10) % Passante da Amostra"] is not None else None,
                        "passerby_sample_n40": a["(ASTM Nº40) % Passante da Amostra"] if a["(ASTM Nº40) % Passante da Amostra"] is not None else None,
                        "passerby_sample_n80": a["(ASTM Nº80) % Passante da Amostra"] if a["(ASTM Nº80) % Passante da Amostra"] is not None else None,
                        "passerby_sample_n200": a["(ASTM Nº200) % Passante da Amostra"] if a["(ASTM Nº200) % Passante da Amostra"] is not None else None,
                        "irre_long": get_value('Irregularidade Longitudinal (IRI)',occurrence_type,a["Irregularidade Longitudinal (IRI)"].strip()) if a["Irregularidade Longitudinal (IRI)"] is not None else None,
                        "tax_lig_res": a["Taxa de Ligante Residual"] if a["Taxa de Ligante Residual"] is not None else None,
                        "macro_sand": a["Macrotextura - Mancha de areia"] if a["Macrotextura - Mancha de areia"] is not None else None,
                        "micro_brit": a["Microtextura - Pêndulo Britânico"] if a["Microtextura - Pêndulo Britânico"] is not None else None,
                        "rejected": a["Rejeito (Desprendimento de agregados)"] if a["Rejeito (Desprendimento de agregados)"] is not None else None,
                        "vol_empty_low": a["Volume de vazios Inferior"] if a["Volume de vazios Inferior"] is not None else None,
                        "vol_empty_high": a["Volume de vazios Superior"] if a["Volume de vazios Superior"] is not None else None,
                        "required_vol": a["Requisito"] if a["Requisito"] is not None else None,
                        "teor_ligante_asfalt_result": a["Teor de Ligante Asfaltico"] if a["Teor de Ligante Asfaltico"] is not None else None,
                        "granulometria_result": a["Granulometria"] if a["Granulometria"] is not None else None,
                        "empty_volume_result": a["Volume de Vazios"] if a["Volume de Vazios"] is not None else None,
                        "umity_damage_ind_result": a["Danos por Umidade Induzida"] if a["Danos por Umidade Induzida"] is not None else None,
                        "long_wrong_result": a["Irregularidade Longitudinal"] if a["Irregularidade Longitudinal"] is not None else None,
                        "macrotextura_result": a["Macrotextura"] if a["Macrotextura"] is not None else None,
                        "microtextura_result": a["Microtextura"] if a["Microtextura"] is not None else None,
                        "loose_aggregated_result": a["Desprendimento de Agregado"] if a["Desprendimento de Agregado"] is not None else None,
                        "density_result": a["Densidade"] if a["Densidade"] is not None else None,
                        },
                    ))
    except Exception as e:
        print(f'AQUI: Linha da Planilha: {index+2} erro: {e}')
#         print(objects.form_data)
        pass

In [ ]:
objectsEnsaio[0]

In [ ]:
for ensaio in tqdm(objectsEnsaio):
    ensaio.save()
    print(ensaio.number)